# Flux1Schnell on CoreAI

The CoreAI container is started using the instructions described in the shared `README.md` file which uses `podman` or `docker`.

## Python virtual environment setup

To enhance the functionality of the CoreAI environment, we need to install some libraries not pre-installed in the base container but required for this notebook. 

### Create and Activate the Virtual Environment:

Open your terminal or command prompt within the Jupyter notebook. Navigate via `File -> New -> Terminal` then type `./create_myvenv.sh` to run the script.

This script will create a `venv` named `myvenv` in the folder where this notebook is, activate it to install an new Python kernel named "Python (myvenv-Flux1Schnell)".

## Change the Python kernel

Before running the following commands from the `Kernel -> Change Kernel` menu option, **Start python kernel** named `Python (myvenv-Flux1Schnell)` kernel. 

When succesful, you will see the new kernel name listed in the rop right corner of this notebook.

### Install Required Libraries:

Ensure you are in the directory where the Jupyter Notebook and the `myvenv` directory are located. 

In [ ]:
!. ./myvenv/bin/activate; pip install -r requirements.txt

Add `accelerate` to the PATH

In [ ]:
import os
pwd = os.getcwd()
os.environ['PATH'] =  os.path.join(pwd, 'myvenv/bin') + os.pathsep + os.environ['PATH']

! echo $PATH
! which accelerate

**Make sure `HF_HOME` is set BEFORE `transformers` is loaded**

In [ ]:
import os
os.makedirs('HF_HOME', exist_ok=True)
os.environ['HF_HOME'] = 'HF_HOME'

Verify that the `HF_TOKEN` is set before attempting to load the model (this is passed as an environment variable within the `start_CoreAI.sh` script.

In [ ]:
if 'HF_TOKEN' not in os.environ:
    printf("No HF_TOKEN set, will not be able to download the model")
    exit(1)

## Running Flux1Schnell on CoreAI

### Obtain model

In [ ]:
import torch
from diffusers import FluxPipeline

In [ ]:
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
pipe.enable_sequential_cpu_offload()
torch.cuda.empty_cache()

### Image generation

Modify the `prompt`, `seed` and `width` and `height` as required.

In [ ]:
import random

#prompt = "An ocean reflection on starry night with the word CoreAI in the sky"
#prompt = "An galaxy with the word CoreAI spelled with stars"
prompt = "a robot that is full-bodied in shades of teal and blue with yellow accents that has a friendly expression and is waving holding a sign that spells CoreAI"

seed = random.randint(1, 999)
width = 1360
height = 768

image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    height=height,
    width=width,
    generator=torch.Generator("cpu").manual_seed(seed)
).images[0]
image.save("flux-schnell.png")

from IPython.display import Image
Image(filename='flux-schnell.png') 